# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
import numpy as np

In [2]:
scd = True

# Define a submodel

In [3]:
class Rectangle(bu.Model):
    name = 'rectangle'
    
    length = bu.Float(2, GEO=True)
    width = bu.Float(2, GEO=True)

    A = tr.Property
    def _get_A(self):
        return self.length * self.width
    
    ipw_view = bu.View(
        bu.Item('length'),
        bu.Item('width')
    )
    
class Circle(bu.Model):
    name = 'circle'
    
    radius = bu.Float(2, GEO=True)

    A = tr.Property
    def _get_A(self):
        return self.radius**2 * np.pi

    ipw_view = bu.View(
        bu.Item('radius')
    )

In [4]:
class ReinfLayer(bu.Model):
    name = 'layer'
    css = bu.EitherType(options=[('rectangle', Rectangle),
                                 ('circle', Circle)])
    
    A = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_A(self):
        return self.css_.A

    depends_on = ['css_']
    ipw_view = bu.View(
        bu.Item('css'),
        bu.Item('A')
    )

In [5]:
rl = ReinfLayer()
rl.state_change_debug = True
rl.interact()

In [7]:
changes = [('rectangle', 'length', 3),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ]
for css_type, attr, value in changes:
    print('-----------------------')
    rl.reset_state_change()
    print(css_type, attr, value)
    rl.css = css_type
    setattr(rl.css_, attr, value)
    print('A', rl.A)
    print('state_changes', rl.state_change_counter)

-----------------------
rectangle length 3
value_changed <__main__.Rectangle object at 0x7f1bcbc95900> TraitChangeEvent(object=<__main__.Rectangle object at 0x7f1bcbc95900>, name='length', old=5.0, new=3.0)
value_changed <__main__.ReinfLayer object at 0x7f1bd4793c70> Notification from child <__main__.Rectangle object at 0x7f1bcbc95900>
A 6.0
state_changes 2
-----------------------
circle radius 3
parent <__main__.ReinfLayer object at 0x7f1bd4793c70> changing child from child <__main__.Rectangle object at 0x7f1bcbc95900> to <__main__.Circle object at 0x7f1bd4612810>
graph_changed <__main__.ReinfLayer object at 0x7f1bd4793c70> Notification from child <__main__.Circle object at 0x7f1bd4612810>
value_changed <__main__.Circle object at 0x7f1bd4612810> TraitChangeEvent(object=<__main__.Circle object at 0x7f1bd4612810>, name='radius', old=2, new=3.0)
value_changed <__main__.ReinfLayer object at 0x7f1bd4793c70> Notification from child <__main__.Circle object at 0x7f1bd4612810>
A 28.27433388230

# Test a dictionary

In [8]:
class CrossSectionLayout(bu.ModelDict):
    name = 'Cross Section Layout'
    item_type = ReinfLayer
    
    sum_A = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_sum_A(self):
        return sum( rl.A for rl in self.items.values() ) 

    ipw_view = bu.View(
        bu.Item('sum_A', readonly=True)
    )

In [14]:
csl = CrossSectionLayout()
csl.state_change_debug = False
csl['one'] = ReinfLayer()

In [20]:
class Beam(bu.Model):
    csl = bu.Instance(CrossSectionLayout, ())
    tree = ['csl']

In [21]:
b = Beam(csl=csl)
# b.interact()

In [25]:
changes = [('rectangle', 'length', 3),
           ('circle', 'radius', 3),
           ('rectangle', 'length', 5),
           ('circle', 'radius', 5),
           ('rectangle', 'length', 5),
           ]
for css_type, attr, value in changes:
    rl.reset_state_change()
    print('-----------------------')
    rl = b.csl['one']
    print(css_type, attr, value)
    rl.css = css_type
    setattr(rl.css_, attr, value)
    print('A', rl.A)
    print('state_changes', rl.state_change_counter)

-----------------------
rectangle length 3
A 6.0
state_changes 5
-----------------------
circle radius 3
A 28.274333882308138
state_changes 9
-----------------------
rectangle length 5
A 10.0
state_changes 9
-----------------------
circle radius 5
A 78.53981633974483
state_changes 9
-----------------------
rectangle length 5
A 10.0
state_changes 9
